In [1]:
# Imports
import json
import typing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy

### Use business and reviews files from the yelp dataset

In [2]:
business=pd.read_json("C:/yelp/yelp_academic_dataset_business.json", lines=True)
print(business.shape)

(150346, 14)


In [3]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


### Filter on Restaurants

In [4]:
business = business[business['categories'].notnull()]
business = business[business['categories'].str.contains('Restaurant')]
print(business.shape)
business.head()

(52286, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


In [5]:
business['stars'].value_counts()

stars
4.0    13444
3.5    11798
4.5     8360
3.0     7641
2.5     4850
2.0     2911
5.0     1554
1.5     1481
1.0      247
Name: count, dtype: int64

In [6]:
business['state'].value_counts()

state
PA     12644
FL      8732
TN      4353
MO      4248
IN      4150
LA      3641
NJ      3343
AZ      2675
AB      2410
NV      1675
ID      1304
CA      1161
IL       983
DE       962
NC         1
CO         1
HI         1
MT         1
XMS        1
Name: count, dtype: int64

### Focus on top 5 states and cities with the highest number of reviews

In [7]:
grouped_counts = business.groupby(['state', 'city'])['review_count'].sum().reset_index()
sorted_grouped_counts = grouped_counts.sort_values(by='review_count', ascending=False)
sorted_grouped_counts=sorted_grouped_counts.head(5)
sorted_grouped_counts

,state,city,review_count
800,PA,Philadelphia,665749
313,LA,New Orleans,465988
927,TN,Nashville,318560
163,FL,Tampa,293130
266,IN,Indianapolis,242024


In [8]:
top_5_cities = sorted_grouped_counts[['state', 'city']]
top_5_cities

,state,city
800,PA,Philadelphia
313,LA,New Orleans
927,TN,Nashville
163,FL,Tampa
266,IN,Indianapolis


In [9]:
business = business[business[['state', 'city']].isin(top_5_cities.to_dict('list')).all(axis=1)]
business.head()
business.shape

(16440, 14)

In [10]:
size = 1000000
review = pd.read_json("C:/yelp/yelp_academic_dataset_review.json", lines=True,chunksize=size)

### Merge reviews with business on business id

In [11]:
#https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88    
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print("finished creating the dataframe")



308437 reviews
291451 reviews
288343 reviews
280627 reviews
294928 reviews
301178 reviews
278328 reviews
finished creating the dataframe


In [12]:
print(df.shape)
df.head()

(2043292, 18)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35


In [13]:
df.rename(columns={'text':'restaurant_reviews'}, inplace=True)
df.drop('business_id', axis=1, inplace=True)
df.head()

,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,restaurant_reviews,date
0,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35


In [14]:
df_final = df.dropna(subset=['restaurant_reviews'])
print(df_final.shape)
df_final.head()

(2043292, 17)


,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,restaurant_reviews,date
0,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35


### Remove reviews with rating 3 as these are neutral and we want to focus on positive and negative

In [15]:
df_final = df_final[(df_final["review_stars"]!=3)]
df_final.head(20)

,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,restaurant_reviews,date
0,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35
5,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",KQSRUu4Aapl0hG6eu2v8iw,4,This is my favorite bakery in Chinatown! I usu...,2018-03-17 23:47:25
7,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",qUfRCH5NUyRDsJfM6jA5PQ,4,Impression: The good thing is that there are l...,2017-04-20 12:37:09
8,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",gNJNxucGoZ31nlH74EQpPg,4,Best egg tarts? \n\nI was lucky to get them fr...,2014-02-25 14:41:08
9,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",NMOxipsnXc6olWdHYzXiYA,4,Little square cakes of happiness with cute bun...,2012-01-10 02:09:18
10,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",ZO3Hh2lSFWmiPjDj0Wad5w,4,This is my second favorite place for pastries ...,2011-12-06 03:29:07


### Reviews with 0 and 1 are negative, reviews with 4 and 5 stars are positive

In [16]:
df_final.loc[df_final['review_stars'] < 3, 'sentiment'] = 0 
df_final.loc[df_final['review_stars'] > 3, 'sentiment'] = 1
df_final.drop('stars', axis=1, inplace=True)
df_final.head(20)

,name,address,city,state,postal_code,latitude,longitude,review_count,is_open,attributes,categories,hours,user_id,review_stars,restaurant_reviews,date,sentiment
0,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53,1.0
1,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,1.0
2,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57,1.0
3,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57,1.0
4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35,1.0
5,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",KQSRUu4Aapl0hG6eu2v8iw,4,This is my favorite bakery in Chinatown! I usu...,2018-03-17 23:47:25,1.0
7,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",qUfRCH5NUyRDsJfM6jA5PQ,4,Impression: The good thing is that there are l...,2017-04-20 12:37:09,1.0
8,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",gNJNxucGoZ31nlH74EQpPg,4,Best egg tarts? \n\nI was lucky to get them fr...,2014-02-25 14:41:08,1.0
9,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",NMOxipsnXc6olWdHYzXiYA,4,Little square cakes of happiness with cute bun...,2012-01-10 02:09:18,1.0
10,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",ZO3Hh2lSFWmiPjDj0Wad5w,4,This is my second favorite place for pastries ...,2011-12-06 03:29:07,1.0


In [17]:
df_final.sentiment.value_counts()

sentiment
1.0    1430010
0.0     368356
Name: count, dtype: int64

In [18]:
df_final = df_final.reset_index(drop=True)
df_final.head()

,name,address,city,state,postal_code,latitude,longitude,review_count,is_open,attributes,categories,hours,user_id,review_stars,restaurant_reviews,date,sentiment
0,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53,1.0
1,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,1.0
2,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57,1.0
3,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57,1.0
4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35,1.0


In [19]:
def remove_punctuation_and_numbers(text):
    # Use a regular expression to remove punctuation and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)
    return text

In [20]:
# Apply the function to the DataFrame's text column
df_final['reviews_text'] = df_final['restaurant_reviews'].apply(remove_punctuation_and_numbers)
df_final.head()

,name,address,city,state,postal_code,latitude,longitude,review_count,is_open,attributes,categories,hours,user_id,review_stars,restaurant_reviews,date,sentiment,reviews_text
0,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53,1.0,This is nice little Chinese bakery in the hear...
1,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,1.0,This is the bakery I usually go to in Chinatow...
2,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57,1.0,A delightful find in Chinatown Very clean and ...
3,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57,1.0,I ordered a graduation cake for my niece and i...
4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35,1.0,HKSTYLE MILK TEA FOUR STARS\n\nNot quite sure...


### NLTK

In [21]:
# Libraries for Sentiment Analysis
import nltk
from nltk.corpus import stopwords
import re, string
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\psriyapareddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\psriyapareddy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\psriyapareddy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\psriyapareddy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Remove stopwors, do stemming and lemmatization

In [25]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import WhitespaceTokenizer

def stem_lemmatize_remove_stopwords(text):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    tokenized_text = WhitespaceTokenizer().tokenize(text)
    
    filtered_text = [w for w in tokenized_text if not w in stop_words]
    stemmed_text = [stemmer.stem(w) for w in filtered_text]
    lemmatized_text = [lemmatizer.lemmatize(w) for w in stemmed_text]
    
    return lemmatized_text

In [26]:
df_final['cleaned_reviews'] = df_final['reviews_text'].apply(stem_lemmatize_remove_stopwords)

In [27]:
df_final.head()

,name,address,city,state,postal_code,latitude,longitude,review_count,is_open,attributes,categories,hours,user_id,review_stars,restaurant_reviews,date,sentiment,reviews_text,cleaned_reviews
0,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53,1.0,This is nice little Chinese bakery in the hear...,"[thi, nice, littl, chine, bakeri, heart, phila..."
1,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06,1.0,This is the bakery I usually go to in Chinatow...,"[thi, bakeri, i, usual, go, chinatown, they, d..."
2,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57,1.0,A delightful find in Chinatown Very clean and ...,"[a, delight, find, chinatown, veri, clean, kin..."
3,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57,1.0,I ordered a graduation cake for my niece and i...,"[i, order, graduat, cake, niec, came, absolut,..."
4,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35,1.0,HKSTYLE MILK TEA FOUR STARS\n\nNot quite sure...,"[hkstyle, milk, tea, four, star, not, quit, su..."


In [28]:
df_final.to_csv('C:/yelp/df_final_yelp_preprocessing.csv', index=False)

### Save the output to a csv file so that we can run vectorizer and ML models on this file

In [29]:
df_final_reviews = df_final[['name','city','cleaned_reviews','sentiment']]
df_final_reviews.to_csv('C:/yelp/df_final_yelp_review_nltk_preprocessed.csv', index=False)

In [32]:
df_final['name'].nunique()

13097